In [21]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer,BitsAndBytesConfig
import json
from query_to_mql_tt_integrated import system_msg,context,postprocess

In [2]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name())

True
Tesla T4


In [3]:
################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)
# Load the entire model on the GPU 0
device_map = {"": 0}


In [4]:
model_name = "NousResearch/llama-2-7b-chat-hf"
model = AutoModelForCausalLM.from_pretrained(model_name,quantization_config=bnb_config,
    device_map=device_map)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [21]:
#Testing prompt

# question = "what are the sales in africa"
# prompt = f"""[INST]<<SYS>>
# Assume you are a system that detects the month and year in the given secentance, user will give only provide the input in english.
# For example if user gives "top  orders in march 2020", you should only return march 2020.

# <</SYS>>
# {question}
# [/INST]
# [END]"""

In [10]:
def gereneate_output_llama2(prompt):
    #prompt = json.dumps(prompt)
    model_input = tokenizer(prompt, return_tensors="pt").to("cuda")
    #model.eval()
    response = tokenizer.decode(model.generate(**model_input, max_length=4096)[0], skip_special_tokens=True)
    return response

In [6]:
def get_user_question(question):
    return "Based on the following context:\n" + str(context) + "\n return a response for the question:\n" + question + "\n"

In [13]:
def compose_prompt(question):
    return f"""[INST]<<SYS>>
            {system_msg}
            <</SYS>>
            {get_user_question(question)}
            [/INST]
            [END]"""

In [42]:
def post_process(output):
    mql = json.loads((output.split("[END]")[1].split("\n\nReasoning:\n\n")[0]).split(" would be:\n\n")[1])
    reason = output.split("[END]")[1].split("\n\nReasoning:\n\n")[1]
    print("*"*50+"MQL"+"*"*50)
    print(mql)
    print("\n"+"*"*50+"Reason"+"*"*50)
    print(reason)
    return mql,reason

In [47]:
%%time
for i in range(100000):
    if i=="yes":
        print(10)

CPU times: user 6.26 ms, sys: 36 µs, total: 6.29 ms
Wall time: 6.26 ms


In [44]:
question = input("Enter any question:" )
output=gereneate_output_llama2(compose_prompt(question))
post_process(output)

Enter any question:which are the top 10 segments in the basis of discounts
**************************************************MQL**************************************************
{'MEASURE': [{'ENTITY': 'Discount', 'RANK': [{'RANK ADJECTIVE': 'top', 'RANK VALUE': '10'}], 'ADJECTIVE': [], 'TONE': ''}], 'DIMENSION': [{'ENTITY': 'Sub-Category', 'RANK': [{'RANK ADJECTIVE': 'top', 'RANK VALUE': '10'}], 'ADJECTIVE': [], 'TONE': ''}], 'FILTER': [{'ENTITY': 'Consumer', 'RANK': [{'RANK ADJECTIVE': 'top', 'RANK VALUE': '10'}], 'ADJECTIVE': [], 'TONE': ''}], 'DERIVED MEASURE': [{'ENTITY': 'Ratio', 'RANK': [{'RANK ADJECTIVE': 'top', 'RANK VALUE': '10'}], 'ADJECTIVE': [], 'TONE': ''}], 'DATE VARIABLE': [{'ENTITY': 'Order Date', 'DATE RANGE': 'last 1 year', 'CONVERTED TIME ELEMENT': '1 year ago'}]}

**************************************************Reason**************************************************
Based on the context provided, the top 10 segments in the basis of discounts can be identified a

({'MEASURE': [{'ENTITY': 'Discount',
    'RANK': [{'RANK ADJECTIVE': 'top', 'RANK VALUE': '10'}],
    'ADJECTIVE': [],
    'TONE': ''}],
  'DIMENSION': [{'ENTITY': 'Sub-Category',
    'RANK': [{'RANK ADJECTIVE': 'top', 'RANK VALUE': '10'}],
    'ADJECTIVE': [],
    'TONE': ''}],
  'FILTER': [{'ENTITY': 'Consumer',
    'RANK': [{'RANK ADJECTIVE': 'top', 'RANK VALUE': '10'}],
    'ADJECTIVE': [],
    'TONE': ''}],
  'DERIVED MEASURE': [{'ENTITY': 'Ratio',
    'RANK': [{'RANK ADJECTIVE': 'top', 'RANK VALUE': '10'}],
    'ADJECTIVE': [],
    'TONE': ''}],
  'DATE VARIABLE': [{'ENTITY': 'Order Date',
    'DATE RANGE': 'last 1 year',
    'CONVERTED TIME ELEMENT': '1 year ago'}]},
 'Based on the context provided, the top 10 segments in the basis of discounts can be identified as follows:\n\n1. Sub-Category: Top 10 segments in the basis of discounts are Sub-Categories with the highest discount rate.\n2. Segment: Top 10 segments in the basis of discounts are Segments with the highest discount v

**************************************************MQL**************************************************
{'MEASURE': [{'ENTITY': 'Discount', 'RANK': [], 'ADJECTIVE': [], 'TONE': ''}, {'ENTITY': 'Purchase Vol', 'RANK': [], 'ADJECTIVE': [], 'TONE': ''}, {'ENTITY': 'Quantity', 'RANK': [{'RANK ADJECTIVE': 'greater', 'RANK VALUE': '10k'}], 'ADJECTIVE': [], 'TONE': ''}, {'ENTITY': 'Sales', 'RANK': [], 'ADJECTIVE': [], 'TONE': ''}], 'DIMENSION': [{'ENTITY': 'Sub-Category', 'RANK': [], 'ADJECTIVE': [], 'TONE': ''}, {'ENTITY': 'Segment', 'RANK': [], 'ADJECTIVE': [], 'TONE': ''}, {'ENTITY': 'Parts', 'RANK': [], 'ADJECTIVE': [], 'TONE': ''}, {'ENTITY': 'Country', 'RANK': [], 'ADJECTIVE': [], 'TONE': ''}], 'FILTER': [{'ENTITY': 'Consumer', 'RANK': [], 'ADJECTIVE': [], 'TONE': ''}, {'ENTITY': 'Phone', 'RANK': [{'RANK ADJECTIVE': 'greater', 'RANK VALUE': '10k'}], 'ADJECTIVE': [], 'TONE': ''}], 'DERIVED MEASURE': [{'ENTITY': 'Ratio', 'RANK': [{'RANK ADJECTIVE': 'greater', 'RANK VALUE': '10k'}], 'ADJEC

({'MEASURE': [{'ENTITY': 'Discount', 'RANK': [], 'ADJECTIVE': [], 'TONE': ''},
   {'ENTITY': 'Purchase Vol', 'RANK': [], 'ADJECTIVE': [], 'TONE': ''},
   {'ENTITY': 'Quantity',
    'RANK': [{'RANK ADJECTIVE': 'greater', 'RANK VALUE': '10k'}],
    'ADJECTIVE': [],
    'TONE': ''},
   {'ENTITY': 'Sales', 'RANK': [], 'ADJECTIVE': [], 'TONE': ''}],
  'DIMENSION': [{'ENTITY': 'Sub-Category',
    'RANK': [],
    'ADJECTIVE': [],
    'TONE': ''},
   {'ENTITY': 'Segment', 'RANK': [], 'ADJECTIVE': [], 'TONE': ''},
   {'ENTITY': 'Parts', 'RANK': [], 'ADJECTIVE': [], 'TONE': ''},
   {'ENTITY': 'Country', 'RANK': [], 'ADJECTIVE': [], 'TONE': ''}],
  'FILTER': [{'ENTITY': 'Consumer', 'RANK': [], 'ADJECTIVE': [], 'TONE': ''},
   {'ENTITY': 'Phone',
    'RANK': [{'RANK ADJECTIVE': 'greater', 'RANK VALUE': '10k'}],
    'ADJECTIVE': [],
    'TONE': ''}],
  'DERIVED MEASURE': [{'ENTITY': 'Ratio',
    'RANK': [{'RANK ADJECTIVE': 'greater', 'RANK VALUE': '10k'}],
    'ADJECTIVE': [],
    'TONE': ''}],
  '